In [3]:
import json
import gzip
import tarfile
import numpy as np
import datetime, time
import pytz
import pickle

In [9]:
load_data = True

if not load_data:
    
    tar = tarfile.open("data/tweet_data.tar.gz", "r:gz")
    
    avgFollowers = {}
    tweetTimes = {}
    avgRetweets = {}

    pst_tz = pytz.timezone('US/Pacific')
    
    for member in tar.getmembers():
        f = tar.extractfile(member)
        followerData = []
        timeData = []
        retweetData = []
        for line in f.readlines():
            entry = json.loads(line)
            #Follower Data
            followerData.append(entry['author']['followers'])
            #Tweet time Data
            citation = entry['citation_date']
            timeData.append(datetime.datetime.fromtimestamp(citation, pst_tz))
            #Retweet Data
            retweetData.append(entry['metrics']['citations']['total'])
        
        avgFollowers[member.name] = (sum(followerData) / len (followerData))
        avgRetweets[member.name] =  (sum(retweetData) / len(retweetData))
        tweetTimes[member.name] = timeData
        
        f.close()
    
    tar.close()
    
    with open('data/train/part1/part_1_1_data.pkl' , 'wb') as file:
        pickle.dump([avgFollowers, tweetTimes, avgRetweets] , file)
    
else:
    with open('data/train/part1/part_1_1_data.pkl' , 'rb') as file:
        [avgFollowers, tweetTimes, avgRetweets] = pickle.load(file)
    

In [10]:
avgFollowers

{'tweets_#gohawks.txt': 2203.931767444827,
 'tweets_#gopatriots.txt': 1401.8955093016164,
 'tweets_#nfl.txt': 4653.252285502502,
 'tweets_#patriots.txt': 3309.978828415827,
 'tweets_#sb49.txt': 10267.31684948685,
 'tweets_#superbowl.txt': 8858.974662784603}